### CNN on CIFR Assignment:

1.  Please visit this link to access the state-of-art DenseNet code for reference - DenseNet - cifar10 notebook link
2.  You need to create a copy of this and "retrain" this model to achieve 90+ test accuracy. 
3.  You cannot use DropOut layers.
4.  You MUST use Image Augmentation Techniques.
5.  You cannot use an already trained model as a beginning points, you have to initilize as your own
6.  You cannot run the program for more than 300 Epochs, and it should be clear from your log, that you have only used 300 Epochs
7.  You cannot use test images for training the model.
8.  You cannot change the general architecture of DenseNet (which means you must use Dense Block, Transition and Output blocks as mentioned in the code)
9.  You are free to change Convolution types (e.g. from 3x3 normal convolution to Depthwise Separable, etc)
10. You cannot have more than 1 Million parameters in total
11. You are free to move the code from Keras to Tensorflow, Pytorch, MXNET etc. 
12. You can use any optimization algorithm you need. 
13. You can checkpoint your model and retrain the model from that checkpoint so that no need of training the model from first if you lost at any epoch while training. You can directly load that model and Train from that epoch. 

In [1]:
# import keras
# from keras.datasets import cifar10
# from keras.models import Model, Sequential
# from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
# from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
# from keras.layers import Concatenate
# from keras.optimizers import Adam
 

In [2]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf

In [3]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 10
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [4]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

170500096/170498071 [==============================] - 4s 0us/step


In [5]:
X_train.shape

(50000, 32, 32, 3)

In [6]:
X_test.shape

(10000, 32, 32, 3)

In [7]:
import os
import numpy as np
os.environ['PYTHONHASHSEED'] = '0'
import tensorflow as tf
import random as rn
##https://keras.io/getting-started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development
## Have to clear the session. If you are not clearing, Graph will create again and again and graph size will increses. 
## Varibles will also set to some value from before session
tf.keras.backend.clear_session()

## Set the random seed values to regenerate the model.
np.random.seed(0)
rn.seed(0)

In [1]:
# Dense Block
def denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (5,5), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Blosck
def transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output

In [9]:
num_filter = 16
dropout_rate = 0.0
l = 16
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (5,5), use_bias=False ,padding='same')(input)
batch_norm = layers.BatchNormalization()(First_Conv2D)

First_Block = denseblock(batch_norm, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   1200        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 16)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          batch_normalization[0][0]        
_______________________________________________________________________________________

In [11]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import CSVLogger
model_callback = ModelCheckpoint('model.hdf5',save_best_only=True,monitor='val_accuracy',verbose=1,save_weights_only=True)
csv_callback = CSVLogger('training.csv',separator=',',append=True)

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen_train = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True,width_shift_range=0.1,height_shift_range=0.1,rotation_range=20,shear_range=20,horizontal_flip=True,zoom_range=0.2)

In [14]:
datagen_train.fit(X_train)

In [15]:
# https://stackoverflow.com/questions/62948249/feature-wise-center-in-imagedatagenerator
datagen_test = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True)
datagen_test.mean = datagen_train.mean
datagen_test.std = datagen_train.std

<h3>Fitting the model</h3>

<h3>30 epochs</h3>

In [16]:
model.fit(datagen_train.flow(X_train,y_train,batch_size=128),steps_per_epoch=len(X_train)//128,epochs=30,validation_data=datagen_test.flow(X_test,y_test),callbacks=[model_callback,csv_callback])

Epoch 1/30
390/390 [==============================] - ETA: 0s - loss: 1.6890 - accuracy: 0.3772
Epoch 00001: val_accuracy improved from -inf to 0.42900, saving model to model.hdf5
390/390 [==============================] - 68s 175ms/step - loss: 1.6890 - accuracy: 0.3772 - val_loss: 1.5983 - val_accuracy: 0.4290
Epoch 2/30
390/390 [==============================] - ETA: 0s - loss: 1.3781 - accuracy: 0.4982
Epoch 00002: val_accuracy improved from 0.42900 to 0.49380, saving model to model.hdf5
390/390 [==============================] - 65s 167ms/step - loss: 1.3781 - accuracy: 0.4982 - val_loss: 1.5834 - val_accuracy: 0.4938
Epoch 3/30
390/390 [==============================] - ETA: 0s - loss: 1.2190 - accuracy: 0.5605
Epoch 00003: val_accuracy improved from 0.49380 to 0.60270, saving model to model.hdf5
390/390 [==============================] - 65s 167ms/step - loss: 1.2190 - accuracy: 0.5605 - val_loss: 1.1272 - val_accuracy: 0.6027
Epoch 4/30
390/390 [==============================] 

In [ ]:

model.load_weights('model.hdf5')

### 10 epochs

In [17]:
model.fit(datagen_train.flow(X_train,y_train,batch_size=128),steps_per_epoch=len(X_train)//128,epochs=10,validation_data=datagen_test.flow(X_test,y_test),callbacks=[model_callback,csv_callback])

Epoch 1/10
390/390 [==============================] - ETA: 0s - loss: 0.4646 - accuracy: 0.8374
Epoch 00001: val_accuracy did not improve from 0.82230
390/390 [==============================] - 64s 165ms/step - loss: 0.4646 - accuracy: 0.8374 - val_loss: 0.5570 - val_accuracy: 0.8165
Epoch 2/10
390/390 [==============================] - ETA: 0s - loss: 0.4521 - accuracy: 0.8425
Epoch 00002: val_accuracy improved from 0.82230 to 0.84070, saving model to model.hdf5
390/390 [==============================] - 65s 166ms/step - loss: 0.4521 - accuracy: 0.8425 - val_loss: 0.4850 - val_accuracy: 0.8407
Epoch 3/10
390/390 [==============================] - ETA: 0s - loss: 0.4493 - accuracy: 0.8444
Epoch 00003: val_accuracy did not improve from 0.84070
390/390 [==============================] - 65s 166ms/step - loss: 0.4493 - accuracy: 0.8444 - val_loss: 0.5070 - val_accuracy: 0.8315
Epoch 4/10
390/390 [==============================] - ETA: 0s - loss: 0.4453 - accuracy: 0.8446
Epoch 00004: val_

### 20 epochs

In [18]:
model.fit(datagen_train.flow(X_train,y_train,batch_size=128),steps_per_epoch=len(X_train)//128,epochs=20,validation_data=datagen_test.flow(X_test,y_test),callbacks=[model_callback,csv_callback])

Epoch 1/20
390/390 [==============================] - ETA: 0s - loss: 0.4017 - accuracy: 0.8601
Epoch 00001: val_accuracy improved from 0.84070 to 0.85550, saving model to model.hdf5
390/390 [==============================] - 65s 166ms/step - loss: 0.4017 - accuracy: 0.8601 - val_loss: 0.4272 - val_accuracy: 0.8555
Epoch 2/20
390/390 [==============================] - ETA: 0s - loss: 0.3909 - accuracy: 0.8647
Epoch 00002: val_accuracy did not improve from 0.85550
390/390 [==============================] - 64s 165ms/step - loss: 0.3909 - accuracy: 0.8647 - val_loss: 0.5574 - val_accuracy: 0.8255
Epoch 3/20
390/390 [==============================] - ETA: 0s - loss: 0.3891 - accuracy: 0.8626
Epoch 00003: val_accuracy did not improve from 0.85550
390/390 [==============================] - 65s 166ms/step - loss: 0.3891 - accuracy: 0.8626 - val_loss: 0.4666 - val_accuracy: 0.8490
Epoch 4/20
390/390 [==============================] - ETA: 0s - loss: 0.3868 - accuracy: 0.8644
Epoch 00004: val_

### 20 epochs

In [19]:
model.fit(datagen_train.flow(X_train,y_train,batch_size=128),steps_per_epoch=len(X_train)//128,epochs=20,validation_data=datagen_test.flow(X_test,y_test),callbacks=[model_callback,csv_callback])

Epoch 1/20
390/390 [==============================] - ETA: 0s - loss: 0.3248 - accuracy: 0.8850
Epoch 00001: val_accuracy did not improve from 0.87110
390/390 [==============================] - 65s 165ms/step - loss: 0.3248 - accuracy: 0.8850 - val_loss: 0.4247 - val_accuracy: 0.8636
Epoch 2/20
390/390 [==============================] - ETA: 0s - loss: 0.3205 - accuracy: 0.8882
Epoch 00002: val_accuracy did not improve from 0.87110
390/390 [==============================] - 65s 166ms/step - loss: 0.3205 - accuracy: 0.8882 - val_loss: 0.7052 - val_accuracy: 0.8078
Epoch 3/20
390/390 [==============================] - ETA: 0s - loss: 0.3170 - accuracy: 0.8878
Epoch 00003: val_accuracy did not improve from 0.87110
390/390 [==============================] - 65s 166ms/step - loss: 0.3170 - accuracy: 0.8878 - val_loss: 0.4119 - val_accuracy: 0.8660
Epoch 4/20
390/390 [==============================] - ETA: 0s - loss: 0.3160 - accuracy: 0.8902
Epoch 00004: val_accuracy did not improve from 0.

### 20 epochs

In [20]:
model.fit(datagen_train.flow(X_train,y_train,batch_size=128),steps_per_epoch=len(X_train)//128,epochs=20,validation_data=datagen_test.flow(X_test,y_test),callbacks=[model_callback,csv_callback])

Epoch 1/20
390/390 [==============================] - ETA: 0s - loss: 0.2749 - accuracy: 0.9023
Epoch 00001: val_accuracy did not improve from 0.88400
390/390 [==============================] - 65s 166ms/step - loss: 0.2749 - accuracy: 0.9023 - val_loss: 0.4072 - val_accuracy: 0.8712
Epoch 2/20
390/390 [==============================] - ETA: 0s - loss: 0.2737 - accuracy: 0.9030
Epoch 00002: val_accuracy did not improve from 0.88400
390/390 [==============================] - 65s 165ms/step - loss: 0.2737 - accuracy: 0.9030 - val_loss: 0.4531 - val_accuracy: 0.8642
Epoch 3/20
390/390 [==============================] - ETA: 0s - loss: 0.2728 - accuracy: 0.9039
Epoch 00003: val_accuracy did not improve from 0.88400
390/390 [==============================] - 65s 166ms/step - loss: 0.2728 - accuracy: 0.9039 - val_loss: 0.3731 - val_accuracy: 0.8832
Epoch 4/20
390/390 [==============================] - ETA: 0s - loss: 0.2655 - accuracy: 0.9067
Epoch 00004: val_accuracy did not improve from 0.

### 50 epochs

In [21]:
model.fit(datagen_train.flow(X_train,y_train,batch_size=128),steps_per_epoch=len(X_train)//128,epochs=50,validation_data=datagen_test.flow(X_test,y_test),callbacks=[model_callback,csv_callback])

Epoch 1/50
390/390 [==============================] - ETA: 0s - loss: 0.2367 - accuracy: 0.9165
Epoch 00001: val_accuracy did not improve from 0.89040
390/390 [==============================] - 64s 165ms/step - loss: 0.2367 - accuracy: 0.9165 - val_loss: 0.3914 - val_accuracy: 0.8793
Epoch 2/50
390/390 [==============================] - ETA: 0s - loss: 0.2363 - accuracy: 0.9168
Epoch 00002: val_accuracy did not improve from 0.89040
390/390 [==============================] - 64s 165ms/step - loss: 0.2363 - accuracy: 0.9168 - val_loss: 0.5208 - val_accuracy: 0.8497
Epoch 3/50
390/390 [==============================] - ETA: 0s - loss: 0.2346 - accuracy: 0.9168
Epoch 00003: val_accuracy did not improve from 0.89040
390/390 [==============================] - 65s 166ms/step - loss: 0.2346 - accuracy: 0.9168 - val_loss: 0.3894 - val_accuracy: 0.8843
Epoch 4/50
390/390 [==============================] - ETA: 0s - loss: 0.2359 - accuracy: 0.9173
Epoch 00004: val_accuracy did not improve from 0.

### 50 epochs

In [23]:
model.load_weights('model.hdf5')
model.compile(optimizer=Adam(learning_rate=0.0001,epsilon=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])

In [24]:
model.fit(datagen_train.flow(X_train,y_train,batch_size=128),steps_per_epoch=len(X_train)//128,epochs=50,validation_data=datagen_test.flow(X_test,y_test),callbacks=[model_callback,csv_callback])

Epoch 1/50
390/390 [==============================] - ETA: 0s - loss: 0.1349 - accuracy: 0.9519
Epoch 00001: val_accuracy improved from 0.90380 to 0.91010, saving model to model.hdf5
390/390 [==============================] - 67s 171ms/step - loss: 0.1349 - accuracy: 0.9519 - val_loss: 0.3191 - val_accuracy: 0.9101
Epoch 2/50
390/390 [==============================] - ETA: 0s - loss: 0.1345 - accuracy: 0.9525
Epoch 00002: val_accuracy improved from 0.91010 to 0.91170, saving model to model.hdf5
390/390 [==============================] - 65s 168ms/step - loss: 0.1345 - accuracy: 0.9525 - val_loss: 0.3086 - val_accuracy: 0.9117
Epoch 3/50
390/390 [==============================] - ETA: 0s - loss: 0.1273 - accuracy: 0.9548
Epoch 00003: val_accuracy did not improve from 0.91170
390/390 [==============================] - 65s 167ms/step - loss: 0.1273 - accuracy: 0.9548 - val_loss: 0.3201 - val_accuracy: 0.9082
Epoch 4/50
390/390 [==============================] - ETA: 0s - loss: 0.1264 - ac

In [33]:
model.load_weights('model.hdf5')
test_loss,test_accuracy = model.evaluate(datagen_test.flow(X_test,y_test))

313/313 [==============================] - 6s 19ms/step - loss: 0.3158 - accuracy: 0.9159


In [35]:
print("test_loss : {:.2f}".format(test_loss))
print("test_accuracy : {:.2f} %".format(test_accuracy*100))

test_loss : 0.32
test_accuracy : 91.59 %
